In [2]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon


In [21]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(url)

spacex_df.head(5)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [ ]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()    # .first() takes the first row from each group
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [23]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]

site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [28]:
# Create a orange circle at NASA Johnson Space Center's coordinate
circle = folium.Circle(
    nasa_coordinate, 
    radius=1000, 
    color='#d35400', 
    fill=True
    ).add_child(folium.Popup('NASA Johnson Space Center'))

# a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
)


# Adding to the Map
site_map.add_child(circle)
site_map.add_child(marker)

In [31]:
# Add a circle and marker for each launch site
for idx, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    
    circle = folium.Circle(
        coordinate,
        radius=1000,
        color='#000000',
        fill=True
    ).add_child(folium.Popup(row['Launch Site']))
    site_map.add_child(circle)
    
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
        )
    )
    
    site_map.add_child(marker)
    
site_map

In [35]:
# Add markers for each launch, colored by success (green) or failure (red)
for _, launch in spacex_df.iterrows():
    launch_coord = [launch['Lat'], launch['Long']]
    color = 'red' if launch['class'] == 1 else 'red'
    
    folium.CircleMarker(
        location=launch_coord,
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"{launch['Launch Site']} - {'Success' if launch['class'] == 1 else 'Failure'}"
    ).add_to(site_map)
    
site_map

In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [ ]:
marker_cluster = MarkerCluster()
#MarkerCluster object from the Folium library
#used to manage large numbers of markers on a map efficiently

In [ ]:
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

In [ ]:
site_map.add_child(marker_cluster)

# For each row in spacex_df data frame, create a Marker object with its coordinate
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=f"{record['Launch Site']} - {'Success' if record['class'] == 1 else 'Failure'}"
    )
    marker_cluster.add_child(marker)

site_map

In [ ]:
from geopy.distance import geodesic

# Calculate distances from each launch site to all other launch sites
distances = []
for i, site1 in launch_sites_df.iterrows():
    coord1 = (site1['Lat'], site1['Long'])
    
    for j, site2 in launch_sites_df.iterrows():
        if i != j:
            coord2 = (site2['Lat'], site2['Long'])
            distance_km = geodesic(coord1, coord2).kilometers
            
            distances.append({
                'From': site1['Launch Site'],
                'To': site2['Launch Site'],
                'Distance_km': distance_km
            })

distances_df = pd.DataFrame(distances)
distances_df

,From,To,Distance_km
0,CCAFS LC-40,CCAFS SLC-40,0.112244
1,CCAFS LC-40,KSC LC-39A,6.911215
2,CCAFS LC-40,VAFB SLC-4E,3833.066450
3,CCAFS SLC-40,CCAFS LC-40,0.112244
4,CCAFS SLC-40,KSC LC-39A,6.946254
5,CCAFS SLC-40,VAFB SLC-4E,3833.080950
6,KSC LC-39A,CCAFS LC-40,6.911215
7,KSC LC-39A,CCAFS SLC-40,6.946254
8,KSC LC-39A,VAFB SLC-4E,3826.267370
9,VAFB SLC-4E,CCAFS LC-40,3833.066450


In [ ]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [16]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [17]:
# Coordinates of the closest coastline points for each launch site (obtained using MousePosition)
coastline_points = {
    'CCAFS LC-40': (28.56367, -80.57163),
    'CCAFS SLC-40': (28.56367, -80.57163),
    'KSC LC-39A': (28.58018, -80.64426),
    'VAFB SLC-4E': (34.63283, -120.62509)
}

# Add distance markers for each launch site to its closest coastline
for idx, row in launch_sites_df.iterrows():
    launch_site = row['Launch Site']
    launch_coord = (row['Lat'], row['Long'])
    
    # Get coastline coordinate
    coastline_coord = coastline_points.get(launch_site, None)
    if coastline_coord:
        # Calculate distance
        distance = calculate_distance(launch_coord[0], launch_coord[1], 
                                    coastline_coord[0], coastline_coord[1])
        
        # Create line between launch site and coastline
        line = folium.PolyLine(
            locations=[launch_coord, coastline_coord],
            weight=1,
            color='blue'
        )
        site_map.add_child(line)
        
        # Create marker at coastline with distance
        distance_marker = folium.Marker(
            coastline_coord,
            icon=DivIcon(
                icon_size=(20,20),
                icon_anchor=(0,0),
                html=f'<div style="font-size: 12; color:#d35400;"><b>{distance:.2f} KM</b></div>',
            )
        )
        site_map.add_child(distance_marker)

# Display the map
site_map

In [18]:
# Add PolyLines and distance markers for each launch site
for idx, row in launch_sites_df.iterrows():
    launch_site = row['Launch Site']
    launch_coord = [row['Lat'], row['Long']]
    
    if launch_site in coastline_points:
        coastline_coord = coastline_points[launch_site]
        
        # Calculate the distance between launch site and coastline
        distance = calculate_distance(
            launch_coord[0], launch_coord[1],
            coastline_coord[0], coastline_coord[1]
        )
        
        # Create PolyLine between launch site and coastline
        coordinates = [launch_coord, list(coastline_coord)]
        lines = folium.PolyLine(
            locations=coordinates,
            weight=1,
            color='blue'
        )
        site_map.add_child(lines)
        
        # Add distance marker at the midpoint
        midpoint = [
            (launch_coord[0] + coastline_coord[0])/2,
            (launch_coord[1] + coastline_coord[1])/2
        ]
        
        distance_marker = folium.Marker(
            midpoint,
            icon=DivIcon(
                icon_size=(150,30),
                icon_anchor=(75,15),
                html=f'<div style="font-size: 12; color:#d35400;"><b>{distance:.2f} KM</b></div>'
            )
        )
        site_map.add_child(distance_marker)

# Display the map
site_map

In [19]:
# Coordinates obtained using MousePosition for each point of interest
points_of_interest = {
    'CCAFS LC-40': {
        'city': ('Cape Canaveral', 28.4017, -80.6047),
        'railway': ('Railway', 28.5722, -80.5853),
        'highway': ('Highway', 28.5632, -80.5708)
    },
    'CCAFS SLC-40': {
        'city': ('Cape Canaveral', 28.4017, -80.6047),
        'railway': ('Railway', 28.5722, -80.5853),
        'highway': ('Highway', 28.5632, -80.5708)
    },
    'KSC LC-39A': {
        'city': ('Titusville', 28.6122, -80.8076),
        'railway': ('Railway', 28.5733, -80.6539),
        'highway': ('Highway', 28.5730, -80.6554)
    },
    'VAFB SLC-4E': {
        'city': ('Lompoc', 34.6391, -120.4579),
        'railway': ('Railway', 34.7380, -120.5743),
        'highway': ('Highway', 34.7040, -120.5638)
    }
}

# Colors for different types of lines
line_colors = {
    'city': 'green',
    'railway': 'orange',
    'highway': 'purple'
}

# Add lines and markers for each point of interest
for idx, row in launch_sites_df.iterrows():
    launch_site = row['Launch Site']
    launch_coord = [row['Lat'], row['Long']]
    
    if launch_site in points_of_interest:
        for poi_type, poi_data in points_of_interest[launch_site].items():
            poi_name, poi_lat, poi_lon = poi_data
            poi_coord = [poi_lat, poi_lon]
            
            # Calculate distance
            distance = calculate_distance(
                launch_coord[0], launch_coord[1],
                poi_coord[0], poi_coord[1]
            )
            
            # Create PolyLine
            coordinates = [launch_coord, poi_coord]
            line = folium.PolyLine(
                locations=coordinates,
                weight=2,
                color=line_colors.get(poi_type, 'gray'),
                dash_array='5, 5' if poi_type == 'railway' else None
            )
            site_map.add_child(line)
            
            # Add marker at the point of interest
            folium.Marker(
                poi_coord,
                icon=folium.Icon(color='red', icon='info-sign'),
                popup=f"{poi_name}<br>Distance: {distance:.2f} km"
            ).add_to(site_map)
            
            # Add distance label at midpoint
            midpoint = [
                (launch_coord[0] + poi_coord[0])/2,
                (launch_coord[1] + poi_coord[1])/2
            ]
            
            folium.Marker(
                midpoint,
                icon=DivIcon(
                    icon_size=(150,30),
                    icon_anchor=(75,15),
                    html=f'<div style="font-size: 12; color:{line_colors.get(poi_type, "gray")};">'
                         f'<b>{distance:.2f} km to {poi_name}</b></div>'
                )
            ).add_to(site_map)

# Display the map
site_map